In [354]:
import operator

import matplotlib.pyplot as plt
%matplotlib inline
import re
from collections import defaultdict

import numpy as np

with open('03/input.txt') as f:
    lines = f.readlines()

In [370]:
part_1 = 0

SIZE = 140


def convert(a):
    if a.isdigit():
        return int(a)
    if a == '.':
        return -50
    return 50


N = np.zeros((SIZE, SIZE))

data = lines[0:SIZE]

for y, line in enumerate(data):
    for x, char in enumerate(line.rstrip()):
        N[y, x] = convert(char)

plt.figure(figsize=(SIZE, SIZE))
plt.imshow(N, cmap='Blues')

for y in range(10):
    for x in range(10):
        if 0 <= N[y, x] < 10:
            plt.text(x, y, str(int(N[y, x])), ha="center", va="center", color="black", size="xx-large")

plt.axis('off')

links = defaultdict(list)

for y, line in enumerate(lines):
    # line.replace('.', ' ')
    str_part_numbers = re.findall(r"\d+", line)
    next_offset = 0
    for num in str_part_numbers:
        part_index = line[next_offset:].index(num) + next_offset
        next_offset = part_index + len(num)
        left = max(0, part_index - 1)
        right = min(SIZE - 1, part_index + len(num) + 1)
        top = max(0, y - 1)
        bottom = min(SIZE - 1, y + 1)
        symbols = set(lines[top][left: right] + line[left: right] + lines[bottom][left: right]) - set('.0123456789')
        if len(symbols) > 0:
            part_1 += int(num)
            if '*' in symbols:
                for _x in range(left, right):
                    for _y in range(top, bottom + 1):
                        if lines[_y][_x] == '*':
                            links[(_x, _y)].append(int(num))

                        char = lines[_y][_x]
                        if char == '.':
                            char = 'X'

                        plt.text(_x, _y, lines[_y][_x], ha="center", va="center", backgroundcolor="pink", color="white",
                                 size="xx-large")

            for x, char in enumerate(num):
                plt.text(part_index + x, y, char, ha="center", va="center", backgroundcolor="gray", color="black",
                         size="xx-large")
        else:
            for x, char in enumerate(num):
                plt.text(part_index + x, y, char, ha="center", va="center", backgroundcolor="red", color="white",
                         size="xx-large")

print(part_1)  # 539590

for link in [link for link in links if len(links[link]) == 2]:
    plt.text(link[0], link[1], '*', ha="center", va="center", backgroundcolor="green", color="black", size="xx-large")

links_parts = [links[link] for link in links if len(links[link]) == 2]
prod_links_parts = [prod[0] * prod[1] for prod in links_parts]

part_2 = sum(prod_links_parts)
print(part_2)
# 78858572 fail

539590
80703636


### Day 4

Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11

In [427]:
with open('04/input.txt') as f:
    lines = f.readlines()

cards = [card.rstrip().split(':')[1].split('|') for card in lines]
copies = [1 for i, _ in enumerate(cards)]

part_1 = 0
for i, card in enumerate(cards):
    win_nums = re.findall(r"\d+", card[0])
    player_nums = re.findall(r"\d+", card[1])
    intersections = list(set(win_nums) & set(player_nums))
    result = 1 if len(intersections) > 0 else 0
    for j, score in enumerate(intersections):
        result = result << 1
        copies[i + j + 1] += copies[i]
    result = result >> 1
    part_1 += result

print('part 1:', part_1)  # 25010

part_2 = sum(copies)
print('part 2:', part_2)  # 9924412

part 1: 25010
part 2: 9924412


### Day 5

seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4

Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

In [625]:
import portion as P

with open('05/input.txt') as f:
    lines = f.readlines()

data = {
    'seeds': [],
    'seed-to-soil map': [],
    'soil-to-fertilizer map': [],
    'fertilizer-to-water map': [],
    'water-to-light map': [],
    'light-to-temperature map': [],
    'temperature-to-humidity map': [],
    'humidity-to-location map': [],
}

states = [s for s in data.keys()]
current_state = None

# parse data
for line in lines:
    line = line.strip()
    if len(line) == 0:
        current_state = None
        continue
    label = line.split(':')[0]
    if label in states:
        current_state = label
        if current_state != 'seeds':
            continue
    match current_state:
        case 'seeds':
            data[current_state] = [int(seed) for seed in line.split(':')[1].strip().split(' ')]
        case _:
            link_info = [int(val) for val in line.strip().split(' ')]
            link_info.append(P.closed(link_info[1], link_info[1] + link_info[2] - 1))
            data[current_state].append(link_info)


def find_min_location(ranges_from):
    for state in states[1:]:
        translated = P.empty()
        for dst, src, r, interval in data[state]:
            intersections = ranges_from & interval
            for intersection in intersections:
                translated |= P.closed(dst + intersection.lower - src, dst + intersection.upper - src)
            ranges_from -= interval 
        ranges_from |= translated
    return ranges_from.lower


# Part I
seeds_intervals = P.empty()
for src in data['seeds']:
    seeds_intervals |= P.closed(src, src)

print('Part 1:', find_min_location(seeds_intervals))


# Part II
new_seeds = np.array(data['seeds'])
new_seeds = new_seeds.reshape(-1, 2)

seeds_intervals = P.empty()
for src, r in new_seeds:
    seeds_intervals |= P.closedopen(src, src + r)

print('Part 2:', find_min_location(seeds_intervals))


Part 1: 3374647
Part 2: 6082852
